# skiplist-06

This version treats the skiplist as if it were totally ordered, so that all nodes are comparable to all nodes because all nodes have token position values for all witness at all times. Specifically:

1. When we create a new skiplist node, we add the actual witness that motivated its creation with the real token position as its value, and we add all other witnesses with `-1` values. This means that the node will always be to the right of the start node, since the start node has `-1` as the value for all witnesses, and data nodes will have a value other than `-1` for at least one witness.
1. Adding a witness to a skiplist node means that the node must already contain a `-1` value for that witness, which we replace with the real token position when we add it. Previously we required that the witness siglum not already be on the node, but now all sigla will always already be on all nodes.
1. We should never get a no-shared-sigla error since all sigla for all witnesses will always be on all nodes.
1. **From a computational perspective, the skiplist will be totally ordered.** This means that we no longer need to filter out nodes before we perform a comparison.
1. **From a human perspective the skiplist remains partially ordered,** since the default `-1` values arbitrarily look left when they have no data-oriented reason to decide which way to go.
1. The comparison operations must look at non-magic-values first (that is, nodes that have shared sigla), instead of always looking at all witnesses (whether present or not) in the same order.
1. We can remove the filtering from all operations except computing the floor and ceiling (and perhaps even from there, too).

TODO: Do we need filtering even with floor and ceiling, or will the default values for missing witnesses (on a node) handle that situation for us?

In [1]:
%load_ext autoreload
%autoreload 2

import pprint, typing
pp = pprint.PrettyPrinter(indent=2)

In [2]:
%run create-blocks.ipynb # data will be in prioritized_blocks
# pp.pprint(prioritized_blocks) # take a look
# pp.pprint(token_to_block_dict) # and another look
# print(token_array, len(token_array)) # and one more look

# Skiplist of variant graph nodes kept in a partial order

Keys are variant graph nodes. Values are not really relevant, but for now they are offsets into the witnesses.
The floor and ceiling should contain not only the siglum of the token we want to insert but also of another witness.
The different methods in the skiplist should no longer try and catch partial order exceptions. Instead filters should be passed along before an insert-and-find operation next to a comparator function so that comparison never fails.

There are two lookups, which happen in the following order:

1. To see whether a token has already been inserted, look only at tokens with the siglum we are processing. For example, if we are placing a token for `w0`, we look at nodes that contain tokens for `w0` (regardless of whether other witnesses are also present on them). If our token is already there, break. We can implement this as a bitarray the length of the `token_array`, initialized to all False.
1. If the token has not already been placed, we next look for a floor and ceiling in order to insert the token. In this case, we look only at nodes that have both of the following properties: a) then contain a token for the witness we are trying to place, and b) they contain tokens for a total of two or more witnesses, that is, at least one witness in addition to the one on which we are focused. For the example above, if we reach this stage, floor and ceiling candidates must contain `w0` and a total of at least 2 sigla. The comparator should not raise an exception except if there is an actual error in the comparison logic (that is, unlike our earlier try/catch approach).

We control step 2 with filters, which we pass to the comparator functions.

In [3]:
#
# This file is part of Bluepass. Bluepass is Copyright (c) 2012-2014
# Geert Jansen.
#
# Bluepass is free software available under the GNU General Public License,
# version 3. See the file LICENSE distributed with this file for the exact
# licensing terms.

from __future__ import absolute_import, print_function

import os
import sys
import math
import random

__all__ = ['SkipList']


# The following functions are debugging functions. They are available only when
# Python is not started with -O.

if __debug__:

    def fmtnode(node):
        """Format a single skiplist node."""
        level = max(1, len(node) - 3)
        skip = '(none)' if level == 1 else node[-1]
        return '<Node(level={}, key={}, value={}, skip={})>' \
                    .format(level, node[0], node[1], skip)

    def dump(sl, file=sys.stdout):
        """Dump a skiplist to standard output."""
        print('== Dumping skiplist {0!r}'.format(sl), file=file)
        print('Level: {}/{}'.format(sl.level, sl.maxlevel), file=file)
        print('Size: {}'.format(len(sl)), file=file)
        node = sl._head
        print('{0} (head)'.format(fmtnode(node)), file=file)
        node = node[2]
        avglvl = avglen = avgsiz = 0
        while node is not sl._tail:
            print('{0}'.format(fmtnode(node)), file=file)
            node = node[2]
            avglvl += max(1, len(node) - 3)
            avglen += len(node)
            avgsiz += nodesize(node)
        print('{0} (tail)'.format(fmtnode(node)), file=file)
        print('Avg level: {:.2f}'.format(avglvl/len(sl)), file=file)
        print('Avg node len: {:.2f}'.format(avglen/len(sl)), file=file)
        print('Avg node memory: {:.2f}'.format(avgsiz/len(sl)), file=file)
        print(file=file)

    def check(sl):
        """Check the internal structure of a skiplist."""
        level = sl.maxlevel
        assert level > 0
        while sl._head[1+level] is sl._tail and level > 1:
            level -= 1
        assert level == sl.level
        assert sl._head[0] is sl._head[1] is None
        assert sl._head[-1] == 0
        pos = 0
        node = sl._head
        inbound = {id(sl._head): 0, id(sl._tail): len(sl)}
        while node is not sl._tail:
            assert isinstance(node, list)
            level = min(sl.level, max(1, len(node)-3))
            assert 1 <= level <= sl.maxlevel
            for i in range(1, level):
                fnode = node[2+i]
                flevel = min(sl.level, max(1, len(fnode)-3))
                if i == flevel-1:
                    inbound[id(fnode)] = pos
            if level > 1:
                assert id(node) in inbound
                assert pos == inbound[id(node)] + node[-1]
            for i in range(level):
                fnode = node[2+i]
                assert isinstance(fnode, list)
                level = max(1, len(node) - 3)
                assert level >= i+1
            node = node[2]
            pos += 1
        assert sl._tail[0] is None
        assert sl._tail[1] is None
        for i in range(sl.maxlevel):
            assert sl._tail[2+i] is None
        assert len(sl) == inbound[id(sl._tail)] + node[-1]

    def nodesize(node):
        """Return the size of a skiplist node."""
        size = sys.getsizeof(node)
        size += sys.getsizeof(node[0])
        size += sys.getsizeof(node[1])
        # All elements in [3:-1] are references so don't count
        if len(node) > 3:
            size += sys.getsizeof(node[-1])
        return size

    def getsize(sl):
        """Return total size of a skiplist."""
        size = sys.getsizeof(sl)
        size += sys.getsizeof(sl._level)
        node = sl._head
        while node is not sl._tail:
            size += nodesize(node)
            node = node[2]
        size += nodesize(node)
        size += sys.getsizeof(sl._path)  # contains references or None
        size += sys.getsizeof(sl._distance)
        for el in sl._distance:
            size += sys.getsizeof(el)
        return size


class SkipList(object):
    """An indexable skip list.

    A SkipList provides an ordered sequence of key-value pairs. The list is
    always sorted on key and supports O(1) forward iteration. It has O(log N)
    time complexity for key lookup, pair insertion and pair removal anywhere in
    the list. The list also supports O(log N) element access by position.

    The keys of all pairs you add to the skiplist must be be comparable against
    each other, and define the ``<`` and ``<=`` operators.
    """

    UNSET = object()

    p = int((1<<31) / math.e)
    maxlevel = 20

    # Kudos to http://pythonsweetness.tumblr.com/post/45227295342 for some
    # useful tricks, including using a list for the nodes to save memory.

    # Use the built-in Mersenne Twister random number generator. It is more
    # appropriate than SystemRandom because we don't need cryptographically
    # secure random numbers, and we don't want to do a system call to read
    # /dev/urandom for each random number we need (every insertion needs a new
    # random number).

    _rnd = random.Random()
    _rnd.seed(os.urandom(16))

    __slots__ = ('_level', '_head', '_tail', '_path', '_distance', '_last_comparable_node_higher')

#####
# Add _last-comparable_node property in __init__(); needed for floor() and 
# ceiling() (add to __slots__, too)
#####
    def __init__(self):
        self._level = 1
        self._head = self._new_node(self.maxlevel, None, None)
        self._tail = self._new_node(self.maxlevel, None, None)
        for i in range(self.maxlevel):
            self._head[2+i] = self._tail
        self._path = [None] * self.maxlevel
        self._distance = [None] * self.maxlevel

    def _new_node(self, level, key, value):
        # Node layout: [key, value, next*LEVEL, skip?]
        # The "skip" element indicates how many nodes are skipped by the
        # highest level incoming link.
        if level == 1:
            return [key, value, None]
        else:
            return [key, value] + [None]*level + [0]

    def _random_level(self):
        # Exponential distribution as per Pugh's paper.
        l = 1
        maxlevel = min(self.maxlevel, self.level+1)
        while l < maxlevel and self._rnd.getrandbits(31) < self.p:
            l += 1
        return l

    def _create_node(self, key, value):
        # Create a new node, updating the list level if required.
        level = self._random_level()
        if level > self.level:
            self._tail[-1] = len(self)
            self._level = level
            self._path[level-1] = self._head
            self._distance[level-1] = 0
        return self._new_node(level, key, value)

    def _find_lt(self, key, skiplist_filter: typing.Callable= lambda x: True):
        # Find path to last node < key
        node = self._head
        distance = 0
        for i in reversed(range(self.level)):
#####
# Keys are partially ordered, so if two are not comparable (because they
# share no sigla), the comparator in S_Key will raise a ComparisonError,
# which we'll trap and then skip to the next key)
#
# We add and use last_comparable_node for our floor() and ceiling(), so
# that we don't return a node that is unordered with respect to the one we
# are querying
#
# Same modification in _find_lt() and _find_lte()
#####
            nnode = node[2+i]
            while nnode is not self._tail:
                true_or_false = skiplist_filter(nnode[0])
                if true_or_false:
                    if nnode[0] < key:
                        nnode, node = nnode[2+i], nnode
                        distance += 1 if i == 0 else node[-1]
                        last_comparable_node = node
                    else:
                        break
                else:
                    nnode = nnode[2+i]
                    continue
            self._path[i] = node
            self._distance[i] = distance
        self._last_comparable_node_higher = nnode

    def _find_lte(self, key: str, skiplist_filter: typing.Callable= lambda x: True):
        # TODO: update all calls to this function in include new skiplist_filter
        #   parameter and remove default value
        # Find path to last node <= key
        node = self._head
        distance = 0
        for i in reversed(range(self.level)):
            nnode = node[2+i] # next node
            while nnode is not self._tail:
                # we call the skiplist_filter on the next node
                true_or_false = skiplist_filter(nnode[0])
                if true_or_false:
                    if nnode[0] <= key:
                        nnode, node = nnode[2+i], nnode # move to next node on same level
                        distance += 1 if i == 0 else node[-1] # wtf
                        last_comparable_node = node # last node to pass through filter
                    else:
                        break # break out of while loop, up to for loop (to lower level in skiplist)
                else: # node did not satisfy filter; keep current node and set next node to actual next-next node
                    nnode = nnode[2+i]
                    continue
            # change state of three fields instead of return:
            #   path, distance, last comparable node higher
            #   which one we use depends on caller
            self._path[i] = node # track node visited on each level; equals last node lte target
            self._distance[i] = distance
        self._last_comparable_node_higher = nnode

    def _find_pos(self, pos):
        # Create path to node at pos.
        node = self._head
        distance = 0
        for i in reversed(range(self.level)):
            nnode = node[2+i]
            ndistance = distance + (1 if i == 0 else nnode[-1])
            while nnode is not self._tail and ndistance <= pos:
                nnode, node, distance = nnode[2+i], nnode, ndistance
                ndistance += 1 if i == 0 else nnode[-1]
            self._path[i] = node
            self._distance[i] = distance

    def _insert(self, node):
        # Insert a node in the list. The _path and _distance must be set.
        path, distance = self._path, self._distance
        # Update pointers
        level = max(1, len(node) - 3)
        for i in range(level):
            node[2+i] = path[i][2+i]
            path[i][2+i] = node
        if level > 1:
            node[-1] = 1 + distance[0] - distance[level-1]
        # Update skip counts
        node = node[2]
        i = 2; j = min(len(node) - 3, self.level)
        while i <= self.level:
            while j < i:
                node = node[i]
                j = min(len(node) - 3, self.level)
            node[-1] -= distance[0] - distance[j-1] if j <= level else -1
            i = j+1

    def _remove(self, node):
        # Remove a node. The _path and _distance must be set.
        path, distance = self._path, self._distance
        level = max(1, len(node) - 3)
        for i in range(level):
            path[i][2+i] = node[2+i]
        # Update skip counts
        value = node[1]
        node = node[2]
        i = 2; j = min(len(node) - 3, self.level)
        while i <= self.level:
            while j < i:
                node = node[i]
                j = min(len(node) - 3, self.level)
            node[-1] += distance[0] - distance[j-1] if j <= level else -1
            i = j+1
        # Reduce level if last node on current level was removed
        while self.level > 1 and self._head[1+self.level] is self._tail:
            self._level -= 1
            self._tail[-1] += self._tail[-1] - len(self)
        return value

    # PUBLIC API ...

    @property
    def level(self):
        """The current level of the skip list."""
        return self._level

    def insert(self, key, value, siglum_filter: typing.Callable = lambda x: True):
        """Insert a key-value pair in the list.

        The pair is inserted at the correct location so that the list remains
        sorted on *key*. If a pair with the same key is already in the list,
        then the pair is appended after all other pairs with that key.
        """
        self._find_lte(key, siglum_filter)
        node = self._create_node(key, value)
        self._insert(node)

    def replace(self, key, value):
        """Replace the value of the first key-value pair with key *key*.

        If the key was not found, the pair is inserted.
        """
        self._find_lt(key)
        node = self._last_comparable_node_higher
        if node is self._tail or key < node[0]:
            node = self._create_node(key, value)
            self._insert(node)
        else:
            node[1] = value

    def clear(self):
        """Remove all key-value pairs."""
        for i in range(self.maxlevel):
            self._head[2+i] = self._tail
            self._tail[-1] = 0
        self._level = 1

    def __len__(self):
        """Return the number of pairs in the list."""
        dist = 0
        idx = self.level + 1
        node = self._head[idx]
        while node is not self._tail:
            dist += node[-1] if idx > 2 else 1
            node = node[idx]
        dist += node[-1]
        return dist

    __bool__ = __nonzero__ = lambda self: len(self) > 0

    def __repr__(self):
        return type(self).__name__ + '((' + repr(list(self.items()))[1:-1] + '))'

    def items(self, start=None, stop=None):
        """Return an iterator yielding pairs.

        If *start* is specified, iteration starts at the first pair with a key
        that is larger than or equal to *start*. If not specified, iteration
        starts at the first pair in the list.

        If *stop* is specified, iteration stops at the last pair that is
        smaller than *stop*. If not specified, iteration end with the last pair
        in the list.
        """
        if start is None:
            node = self._head[2]
        else:
            self._find_lt(start)
            node = self._last_comparable_node_higher
        # while node is not self._tail and (stop is None or stop_filter(node[0]) and node[0] < stop):

        # If we reach the tail, we stop
        # Otherwise, does the node pass the stop filter
        # If we get that far, we can check whether the node is less than the stop value …
        #     … and. if it is, we yield
        # If we don't get that far, we advance to the next node and continue the while loop
        while node is not self._tail:
            yield (node[0], node[1])
            if stop:
                if node[0] < stop:
                    node = node[2]
                    continue
                else:
                    break
            node = node[2]

    __iter__ = items

    def keys(self, start=None, stop=None):
        """Like :meth:`items` but returns only the keys."""
        return (item[0] for item in self.items(start, stop))

    def values(self, start=None, stop=None):
        """Like :meth:`items` but returns only the values."""
        return (item[1] for item in self.items(start, stop))

    def popitem(self):
        """Removes the first key-value pair and return it.

        This method raises a ``KeyError`` if the list is empty.
        """
        node = self._head[2]
        if node is self._tail:
            raise KeyError('list is empty')
        self._find_lt(node[0])
        self._remove(node)
        return (node[0], node[1])

    # BY KEY API ...

    def search(self, key, default=None):
        """Find the first key-value pair with key *key* and return its value.

        If the key was not found, return *default*. If no default was provided,
        return ``None``. This method never raises a ``KeyError``.
        """
        self._find_lt(key)
        node = self._last_comparable_node_higher
        if node is self._tail or key < node[0]:
            return default
        return node[1]

    def remove(self, key, siglum_filter: typing.Callable = lambda x: True):
        """Remove the first key-value pair with key *key*.

        If the key was not found, a ``KeyError`` is raised.
        """
        self._find_lt(key, siglum_filter)
        node = self._last_comparable_node_higher
        if node is self._tail or key < node[0]:
            raise KeyError('{!r} is not in list'.format(key))
        self._remove(node)

    def pop(self, key, default=UNSET):
        """Remove the first key-value pair with key *key*.

        If a pair was removed, return its value. Otherwise if *default* was
        provided, return *default*. Otherwise a ``KeyError`` is raised.
        """
        self._find_lt(key)
        node = self._last_comparable_node_higher
        if node is self._tail or key < node[0]:
            if default is self.UNSET:
                raise KeyError('key {!r} not in list')
            return default
        self._remove(node)
        return node[1]

    def __contains__(self, key):
        """Return whether *key* is contained in the list."""
        self._find_lt(key)
        node = self._last_comparable_node_higher
        return node is not self._tail and not key < node[0]

    def index(self, key, default=UNSET):
        """Find the first key-value pair with key *key* and return its position.

        If the key is not found, return *default*. If default was not provided,
        raise a ``KeyError``
        """
        self._find_lt(key)
        node = self._last_comparable_node_higher
        if node is self._tail or key < node[0]:
            if default is self.UNSET:
                raise KeyError('key {!r} not in list'.format(key))
            return default
        return self._distance[0]

    def count(self, key):
        """Return the number of pairs with key *key*."""
        count = 0
        pos = self.index(key, -1)
        if pos == -1:
            return count
        count += 1
        for i in range(pos+1, len(self)):
            if self[i][0] != key:
                break
            count += 1
        return count

    # BY POSITION API ...

    def __getitem__(self, pos):
        """Return a pair by its position.

        If *pos* is a slice, then return a generator that yields pairs as
        specified by the slice.
        """
        size = len(self)
        if isinstance(pos, int):
            if pos < 0:
                pos += size
            if not 0 <= pos < size:
                raise IndexError('list index out of range')
            self._find_pos(pos)
            node = self._path[0][2]
            return (node[0], node[1])
        elif isinstance(pos, slice):
            start, stop = pos.start, pos.stop
            if start is None:
                start = 0
            elif start < 0:
                start += size
            if stop is None:
                stop = size
            elif stop < 0:
                stop += size
            self._find_pos(start)
            def genpairs():
                pos = start; node = self._path[0][2]
                while node is not self._tail and pos < stop:
                    yield (node[0], node[1])
                    node = node[2]; pos += 1
            return genpairs()
        else:
            raise TypeError('expecting int or slice, got {0.__name__!r}'.format(type(pos)))

    def __delitem__(self, pos):
        """Delete a pair by its position."""
        if not isinstance(pos, int):
            raise TypeError('expecting int, got {0.__name__!r}'.format(type(pos)))
        size = len(self)
        if pos < 0:
            pos += size
        if not 0 <= pos < size:
            raise IndexError('list index out of range')
        self._find_pos(pos)
        node = self._path[0][2]
        self._remove(node)

    def __setitem__(self, pos, value):
        """Set a value by its position."""
        if not isinstance(pos, int):
            raise TypeError('expecting int, got {0.__name__!r}'.format(type(pos)))
        size = len(self)
        if pos < 0:
            pos += size
        if not 0 <= pos < size:
            raise IndexError('list index out of range')
        self._find_pos(pos)
        node = self._path[0][2]
        node[1] = value

    # our custom in function with a filter
    # not pretty, but hey... if it works
#     def custom_in_with_filter(self, key, skiplist_filter: typing.Callable):
#         """Return whether *key* is contained in the list."""
#         self._find_lt(key, skiplist_filter)
#         node = self._last_comparable_node_higher
#         return node is not self._tail and not key < node[0]

In [4]:
from dataclasses import dataclass

@dataclass
class SkiplistNode:
    """Declare SkipListNode class for export in dumpNodes()"""
    level: int
    key: str
    value: list
    name: str = None

def getSkiplistNodeData(node):
    level = max(1, len(node) - 3)
    key = node[0]
    value = [node[1]][0]
    return(level, key, value)

def dumpNodes(sl):
    SkiplistNodes = []
    node = sl._head
    nodeValues = getSkiplistNodeData(node)
    SkiplistNodes.append(SkiplistNode(level=nodeValues[0], key=nodeValues[1], value=nodeValues[2], name="head"))
    node = node[2]
    while node is not sl._tail:
        nodeValues = getSkiplistNodeData(node)
        SkiplistNodes.append(SkiplistNode(level=nodeValues[0], key=nodeValues[1], value=nodeValues[2]))
        node = node[2]
    nodeValues = getSkiplistNodeData(node)
    SkiplistNodes.append(SkiplistNode(level=nodeValues[0], key=nodeValues[1], value=nodeValues[2], name="tail"))
    return SkiplistNodes

# def floor(sl: SkipList, key: str) -> str:
#     """Return highest key value that is less than the new key that we seek to place

#     Assumption: keys are unique
#     Note: lt, not le (unlike Java)"""
#     sl._find_lte(key) # change state (eek!)
#     return sl._path[0][0] if sl._path[0] else None

# def ceiling(sl: SkipList, key: str) -> str:
#     """Return lowest value that is greater than the new key that we seek to place

#     Assumption: keys are unique
#     Note: gt, not ge (unlike Java)"""

#     sl._find_lte(key) # change state (eek!)
#     return sl._last_comparable_node_higher[0] if sl._last_comparable_node_higher else None

import typing
def floor_and_ceiling(sl: SkipList, key: str, skiplist_filter: typing.Callable = lambda x: True) -> tuple:
    """Return tuple of (floor, ceiling), both of which are skiplist nodes

    Neither can be None; start and end nodes always satisfy filter
    Assumption: keys are unique
    Note: lt and gt, not le and ge (unlike Java)"""
    sl._find_lte(key, skiplist_filter) # change state (eek!)
    _floor = sl._path[0][0]
    _ceiling = sl._last_comparable_node_higher[0]
    return (_floor, _ceiling)

In [5]:
from functools import total_ordering

# Find all witness that are present on both items (ignore those on only one item)
#     If none, raise an error
#     If some, compare in arbitrary but consistent order
#         If first pair differs, return result
#         If first pair matches, check next pair

class ComparisonError(Exception):
    def __init__(self, message):
        self.message = message
    def __str__(self):
        return self.message

@total_ordering
class VG_node():
    """Variant graph node, used as key in skiplist

    We don't care about the value in the skiplist; the node contains all information"""
    def __init__(self, block_id=None, token_offset=None, sigla:list=[], **kwargs):
        self.__dict__.update({siglum: -1 for siglum in sigla}) # default all to -1
        self.__dict__.update(kwargs) # overwrite the real value
        self._sigla = {key for key in kwargs.keys()} # FIXME: Creating a set undermines the order
        self.block_id = block_id
        self.token_offset = token_offset
    def __repr__(self):
        return "|".join([":".join([str(key), str(getattr(self, key))]) for key in self.sigla()])
    def __setitem__(self, key, value):
        self._sigla.add(key)
        self.__dict__[key] = value
    def __getitem__(self, key):
        return self.__dict__[key]
    def __contains__(self, key):
        return key in self.__dict__
    def sigla(self):
        return self._sigla
    # #####
    # TODO: remove find_shared_sigla() and remove dependency on it from comparisons below
    # #####
    def find_shared_sigla(self, other):
        """Return shared sigla for use in comparison"""
        non_magic_self = set([siglum for siglum in self.sigla() if getattr(self, siglum) != -1])
        non_magic_other = set([siglum for siglum in other.sigla() if getattr(other, siglum) != -1])
        shared_sigla = sorted(non_magic_self.intersection(non_magic_other))
#         if not shared_sigla: # this should never happen
#             raise ComparisonError("No shared sigla; cannot be compared")
#         return shared_sigla
    def __eq__(self, other):
        # Compare shared non-magic sigla with highest priority, but also the rest, with lower priority
        shared_sigla = self.find_shared_sigla(other) # there may or may not be shared non-magic sigla
        if shared_sigla: # are the shared non-magic values equal?
            shared_equal = [getattr(self, siglum) for siglum in shared_sigla] == [getattr(other, siglum) for siglum in shared_sigla]
        else:
            shared_equal = False
        # now compare non-shared sigla in stable order
        # includes cases where one or both values are magic
        non_shared_equal = \
        [getattr(self, siglum) for siglum in self.sigla() if shared_sigla and siglum not in shared_sigla] == \
        [getattr(other, siglum) for siglum in other.sigla() if shared_sigla and siglum not in shared_sigla]
        return shared_equal and non_shared_equal
    def __lt__(self, other):
        shared_sigla = self.find_shared_sigla(other)
        if shared_sigla: # are the shared non-magic values equal?
            if [getattr(self, siglum) for siglum in shared_sigla] > [getattr(other, siglum) for siglum in shared_sigla]:
                return False
            else:
                shared_less_than = [getattr(self, siglum) for siglum in shared_sigla] < [getattr(other, siglum) for siglum in shared_sigla]
        else: # there are no shared sigla, so check non-shared
            shared_less_than = False
        # now compare non-shared sigla in stable order
        # includes cases where one or both values are magic
        print(self.sigla())
        print(shared_sigla)
        non_shared_less_than = \
            [getattr(self, siglum) for siglum in self.sigla() if shared_sigla and siglum not in shared_sigla] < \
            [getattr(other, siglum) for siglum in other.sigla() if shared_sigla and siglum not in shared_sigla]
        return shared_less_than or non_shared_less_than

In [6]:
# we can use token-string frequency (overall) to align rare tokens first within each block
from collections import Counter
all_token_frequencies = Counter(token_array)
# exclude singletons (not in blocks)
token_frequencies = {k:v for k, v in all_token_frequencies.items() if v > 1}
# bitarray keeps track of which tokens we've already placed in skiplist
from bitarray import bitarray
placed_tokens = bitarray(len(token_array))
placed_tokens.setall(0)
# print(placed_tokens) # take a look

In [7]:
# pp.pprint(token_frequencies) # take a look

# Skiplist setup code above, now create vg (variant graph) skiplist below

# Import code to generate SVG of skiplist from render_skiplist_as_svg.ipynb

In [8]:
vg = SkipList()
tail_value = len(token_array) + 1
vg.insert(VG_node(None, None, witness_sigla, w0=-1, w1=-1, w2=-1, w3=-1), ['[START]'])
vg.insert(VG_node(None, None, witness_sigla, w0=tail_value, w1=tail_value, w2=tail_value, w3=tail_value), ['[END]'])

for index, prioritized_block in enumerate(prioritized_blocks):
    # add each token from the block individually
    # first block: Block(token_count=2, start_position=4, end_position=6, all_start_positions=[2, 9, 18], witnesses={0, 1, 2}, witness_count=3, frequency=3)
    if index: # limit number of blocks for testing; open the limit for production (currently breaks at 20)
        print(f'Processing block #{index}')

        #####
        # Change order:
        # Old: first each witness, then each offset by inverse token frequency
        #   Wrong because it places high-frequency token in first witness before low-frequency token in other
        # New: first each offset by inverse frequency, then same offset for each witness
        #####

        all_offsets = range(prioritized_block.token_count)

        #####
        # from the inside out:
        # 1. take an arbitrary start position from the list of all start positions
        # 2. add each offset to it in turn and use that to retrieve a string from token_array
        # 3. use that string to retrieve a frequency from token_frequencies
        # 4. sort the offsets according to the frequency of the associated tokens (from low to high)
        # 5. we will then be able to process the offsets in order of inverse frequency
        #####
        sorted_offsets = sorted(all_offsets, key=lambda x: token_frequencies[token_array[prioritized_block.all_start_positions[0] + x]])
#         pp.pprint(
#             [(token_array[x + prioritized_block.all_start_positions[0]], token_frequencies[token_array[x + prioritized_block.all_start_positions[0]]]) for x in sorted_offsets])

        for sorted_offset in sorted_offsets:
            for start_position in prioritized_block.all_start_positions:
                witness_token_pos = sorted_offset + start_position

                if placed_tokens[witness_token_pos]:
                    continue # already placed, so look at the next token

                placed_tokens[witness_token_pos] = 1 # we're going to place it below

                #####
                # We go from lower to higher frequency, but we do an entire witness at a time, which is wrong.
                # We need, instead, to go from lower to higher frequency but place the token for all witnesses
                #   before moving on to the next token.
                #####

                current_witness = 'w' + str(token_membership_array[witness_token_pos])

                # function to filter skiplist (to be passed into skiplist API)
                # keep nodes that have:
                #   1) the siglum of interest (= current_witness variable)
                #   2) at least two sigla
                siglum_filter = lambda x: current_witness in x.sigla() and len(x.sigla()) > 1

                key_to_check = VG_node(index, sorted_offset, witness_sigla, **{current_witness: witness_token_pos})
                # if not tuple_to_token_map contains (index, token_offset_in_block) put it in there
                # If it is there, check whether it is the same token/string.
                string_to_look_for = token_array[witness_token_pos]

                # print('we are checking ' + str(key_to_check))
                # TODO: check this in the bitarray
                # TODO: we may not need our custom "in" now that we filter for siglum here
                # if not vg.custom_in_with_filter(key_to_check, lambda x: current_witness in x.sigla()):
                if True: # TODO: remove this and fix indentation
                    # print('we need to add  ', str(key_to_check))
                    # floor_node = floor(vg, key_to_check)
                    # print('floor =         ', floor_node)
                    # ceiling_node = ceiling(vg, key_to_check)
                    # print('ceiling =       ', ceiling_node)
                    floor_node, ceiling_node = floor_and_ceiling(vg, key_to_check, siglum_filter)

#                     if index == 19:
#                         print("We are trying to place "+str(key_to_check))
#                         print('floor = ', floor_node)
#                         print('ceiling = ', ceiling_node)
#                         print('What does the skiplist look like at this time?')
#                         for node_position, node in enumerate(vg.keys()):
#                             print('Node position: ' + str(node_position) + ': ' + str(node))

                    # create a list of the items between the floor and ceiling
                    # floor is inclusive, ceiling is exclusive in the skiplist items method. 
                    gen = vg.items(floor_node, ceiling_node)
                    # we have to skip the floor node
                    zzz = next(gen)
                    current_slice = list(gen)
#                     if index == 19:
#                         print(f"{zzz=}")
#                         print(f"{len(current_slice)=}")
                    # print(current_slice)
                    if not len(current_slice): # floor and ceiling are adjacent, so insert between them
                        # insertion position is automatic for a skiplist (if the comparator is doing its job)
                        vg.insert(key_to_check, [witness_token_pos])
                    else: # do stuff
                        # print()
                        # #####
                        # Are any of the slice nodes candidates for placing new token?
                        # Blocks contain information about all occurrences of pattern.
                        # If token is part of the same block, it can be added to it?
                        #
                        # We already did string matching when we constructed blocks. Now
                        # find intersection of:
                        #   1) nodes between floor and ceiling and
                        #   2) instances of block
                        # If there is an intersection, that's the node with which to align.
                        # To do that:
                        #   1) Remove the existing node
                        #   2) Modify its key (and value?) and insert it
                        # We think it will go back into the same place, but if not, the
                        # skiplist should place it correctly (topologically) anyway.
                        # NB: We do not modify the key or value in place!
                        #
                        # Possibilities:
                        #   1) none
                        #      Insert new node
                        #   2) exactly one
                        #      Add to it
                        #   3) more than one (if there is repetition)
                        #      Defer consideration until we encounter one
                        #
                        # print(f"{slice_length=}") # diagnostic
                        #
                        # Is it situation 1, 2, or 3?
                        #
                        # token_to_block_dict is a multi-valued dictionary:
                        #   keys are token offsets
                        #   values are lists of block offsets in  prioritized_blocks list, which serve as block ids
                        for current_node in current_slice: # look at all vg nodes between floor and ceiling; tuple of vg_node, [offsets]
                            # print("We are visiting "+str(current_node)+" as a potential candidate for alignment")
                            # index is for block we're trying to place; is in graph between floor and ceiling?
                            blocks_for_vg_node = [] # blocks associated with current variant graph node
                            for current_token_offset in current_node[1]: # for each node, look at each of its values
                                blocks_for_current_token = token_to_block_dict[current_token_offset] # non-empty list of integers
                                blocks_for_vg_node.extend(blocks_for_current_token) # list may include duplicates; do we care?
                            # print(f"{blocks_for_vg_node=}")
                            #     if token_offset_in_block == current_node[0].token_offset and index in blocks_for_vg_node: # does the current block, inside the slice, contain the node we care about?
                            if current_witness not in current_node[0].sigla() and token_array[current_node[1][0]] == token_array[witness_token_pos]:
#                                 if index == 19:
#                                     print(f'Before remove: {vg=}')
#                                     print(f'Before remove: {current_node[0]=}')
#                                     print(f'{current_witness=}')
#                                     print('new_key_siglum will equal: w' + str(token_membership_array[witness_token_pos]))
                                # vg.remove(current_node[0], lambda x: x == current_node[0]) # found one; remove matching node by key
                                vg.remove(current_node[0]) # found one; remove matching node by key
#                                 if index == 19:
#                                     print(f'After remove: {current_node[0]=}')
#                                     print(f'After remove: {vg=}')
#                                 new_key_siglum = 'w' + str(token_membership_array[witness_token_pos]) # witness siglum for new token instance
#                                 if new_key_siglum in new_key:
#                                     raise Exception("The key is already on this node")
                                current_node[0][current_witness] = witness_token_pos # add new token to key
                                current_node[1].append(witness_token_pos) # add new token to value, too
                                # print(new_key, new_value)

                                # Debug: CHECK whether the token content is the same when adding an offset to an existing node.
#                                 string_already_on_the_node = token_array[current_node[1][0]]
#                                 string_to_be_placed = token_array[witness_token_pos]
#                                 if new_key_siglum == "w0" and string_already_on_the_node != string_to_be_placed:
#                                     raise Exception(string_already_on_the_node+" != "+string_to_be_placed+" Nodes to check: "+str(current_slice)+" floor: "+str(floor_node)+" : "+str(ceiling_node))


                                vg.insert(current_node[0], current_node[1]) # update key and value, insert
                                break
                        else:
                            # didn't find one, insert new; look only at nodes with current siglum
                            vg.insert(key_to_check, [witness_token_pos])

# place tokens that are not parts of blocks
unplaced_tokens = [offset for offset in range(len(token_array))
                   if offset not in token_to_block_dict
                   and isinstance(token_membership_array[offset], int)]
for unplaced_token in unplaced_tokens:
    current_witness = 'w' + str(token_membership_array[unplaced_token])
    new_node = VG_node(None, None, witness_sigla, **{current_witness: unplaced_token})
    vg.insert(new_node, [unplaced_token])

# dumpNodes(vg) # take a look
# pp.pprint(prioritized_blocks) # take a look at this, too

{'w1', 'w0', 'w3', 'w2'}
None
Processing block #1
{'w1', 'w0', 'w3', 'w2'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w1'}
None
{'w1'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w2'}
None
{'w2'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w3'}
None
{'w3'}
None
{'w3'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w3'}
None
{'w2'}
None
{'w3'}
None
{'w3'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w1'}
None
{'w2'}
None
{'w1'}
None
{'w1'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w3'}
None
{'w2'}
None
{'w2'}
None
{'w3'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w1'}
None
{'w2'}
None
{'w2'}
None
{'w1'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w3'}
None
{'w2'}
None
{'w2'}
N

None
{'w1'}
None
{'w3'}
None
{'w3'}
None
{'w3'}
None
{'w1'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w2'}
None
{'w3'}
None
{'w3'}
None
{'w3'}
None
{'w2'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w3'}
None
{'w3'}
None
{'w3'}
None
{'w3'}
None
{'w3'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w1'}
None
{'w3'}
None
{'w3'}
None
{'w3'}
None
{'w1'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w2'}
None
{'w3'}
None
{'w3'}
None
{'w3'}
None
{'w2'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w3'}
None
{'w3'}
None
{'w3'}
None
{'w3'}
None
{'w3'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w1'}
None
{'w3'}
None
{'w3'}
None
{'w1'}
None
{'w1'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w2'}
None
{'w3'}
None
{'w3'}
None
{'w1'}
None
{'w2'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w3'}
None
{'w3'}
None
{'w3'}
None
{'w1'}
None
{'w3'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w1'}
None
{'w3'}
None
{'w3'}
None
{'w1'}
None
{'w1'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w2'}
None
{'w3'}
None
{'w3'}
None
{'w1'}
None
{'w2'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w3'


None
{'w2'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w3'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w3'}
None
{'w3'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w1'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w3'}
None
{'w1'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w3'}
None
{'w2'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w3'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w3'}
None
{'w3'}
None
{'w3'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w1'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w3'}
None
{'w3'}
None
{'w1'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w3'}
None
{'w2'}
None
{'w2'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w3'}
None
{'w2'}
None
{'w2'}
None
{'w3'}
None
{'w3'}
None
{'w3'}
None
{'w3'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w1

None
{'w2'}
None
{'w1'}
None
{'w1'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w1'}
None
{'w2'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w3'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w1'}
None
{'w3'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w1'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w1'}
None
{'w1'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w3'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w3'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w1'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w1'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w3'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w3'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w1'}
None
{'w2'


{'w2'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w3'}
None
{'w2'}
None
{'w2'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w3'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w3'}
None
{'w2'}
None
{'w3'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w3'}
None
{'w2'}
None
{'w1'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w2'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w3'}
None
{'w2'}
None
{'w2'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w3'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w3'}
None
{'w2'}
None
{'w3'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w3'}
None
{'w2'}
None
{'w1'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w2'}
None
{'w1'}
None
{'w1'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w3'}
None
{'w1'}
None
{'w1'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w3'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w1'}
None
{'w

...

None
{'w1', 'w0', 'w3', 'w2'}
None
{'w0'}
None
{'w0'}
None
{'w1'}
None
{'w1'}
None
{'w3'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w1'}
None
{'w0'}
None
{'w1'}
None
{'w1'}
None
{'w3'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w2'}
None
{'w0'}
None
{'w1'}
None
{'w1'}
None
{'w3'}
None
{'w1'}
None
{'w1'}
None
{'w2'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w3'}
None
{'w0'}
None
{'w1'}
None
{'w1'}
None
{'w3'}
None
{'w1'}
None
{'w3'}
None
{'w3'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w0'}
None
{'w0'}
None
{'w1'}
None
{'w1'}
None
{'w3'}
None
{'w1'}
None
{'w3'}
None
{'w0'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w1'}
None
{'w0'}
None
{'w1'}
None
{'w1'}
None
{'w3'}
None
{'w1'}
None
{'w3'}
None
{'w1'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w2'}
None
{'w0'}
None
{'w1'}
None
{'w1'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w3'}
None
{'w0'}
None
{'w1'}
None
{'w1'}
None
{'w2'}
None
{'w2'}
None
{'w2'}
None

None
{'w1', 'w0', 'w3', 'w2'}
None
{'w3'}
None
{'w0'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w3'}
None
{'w3'}
None
{'w3'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w0'}
None
{'w0'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w3'}
None
{'w3'}
None
{'w0'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w1'}
None
{'w0'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w2'}
None
{'w0'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w2'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w3'}
None
{'w0'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w3'}
None
{'w3'}
None
{'w3'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w0'}
None
{'w0'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w3'}
None
{'w3'}
None
{'w0'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w1'}
None
{'w0'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w2'}
None
{'w0'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None
{'w1'}
None


{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
Processing block #51
Processing block #52
Processing block #53
Processing block #54
Processing block #55
Processing block #56
Processing block #57
Processing block #58
Processing block #59
Processing block #60
Processing block #61
Processing block #62
Processing block #63
Processing block #64
Processing block #65
Processing block #66
Processing block #67
{'w1', 'w0', 'w3', 'w2'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None


Processing block #115
Processing block #116
Processing block #117
Processing block #118
Processing block #119
Processing block #120
Processing block #121
Processing block #122
Processing block #123
Processing block #124
Processing block #125
Processing block #126
Processing block #127
Processing block #128
Processing block #129
Processing block #130
Processing block #131
Processing block #132
Processing block #133
Processing block #134
Processing block #135
Processing block #136
Processing block #137
Processing block #138
Processing block #139
Processing block #140
Processing block #141
Processing block #142
Processing block #143
Processing block #144
Processing block #145
Processing block #146
Processing block #147
Processing block #148
Processing block #149
Processing block #150
Processing block #151
Processing block #152
Processing block #153
Processing block #154
Processing block #155
Processing block #156
Processing block #157
Processing block #158
Processing block #159
Processing

{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w1', 'w0', 'w3', 'w2'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
{'w0'}
None
Processing block #198
Processing block #199
Processing block #200
Processing block #201
Processing block #202
Processing block #203
Processing block

In [9]:
%run render_skiplist_as_svg.ipynb

# Import code to generate SVG of variant graph from render_variant_graph_as_svg.ipynb

Rendering instruction in imported module doesn’t work; we need it issue it here

In [10]:
# get ready to visualize the decision tree in SVG
import graphviz
from IPython.display import SVG
from collections import defaultdict

# TODO: use tm instead of to_variant_graph_nodes because reasons
# TODO: add token offsets with witness identifiers to label

# node id values must be strings for graphviz
a = graphviz.Digraph(format="svg")
a.attr(rankdir = "LR")
# #####
# new code starts here
# #####
for index, vg_node in enumerate(dumpNodes(vg)[1:-1]): # list of SkiplistNode objects, omit head and tail
# create nodes
    node_id = str(index)
    block_id = vg_node.key.block_id
    token_string = token_array[vg_node.value[0]] if isinstance(vg_node.value[0], int) else vg_node.value[0]
    all_token_offsets = ",".join([str(x) for x in vg_node.value])
    a.node(node_id, label=(node_id + ':' + token_string + '(' + str(block_id) + ')(' + all_token_offsets + ')'))
# create edges witness by witness
# print([x.key.sigla() for x in dumpNodes(vg)[1:-1]]) # keys for all skiplist nodes are variant-graph nodes, which have a sigla() method
# for w in range(len(witnesses)): # w is offset of witness in list of witnesses
#     siglum = 'w' + str(w) # siglum as recorded in SkiplistNode keys
#     # witness_nodes = filter(lambda x: siglum in x.key.sigla(), dumpNodes(vg)[1:-1]) # this loses the original position, which is the node id in the variant graph
#     # print(list(witness_nodes)) take a look
#     witness_nodes = []
#     for index, vg_node in enumerate(dumpNodes(vg)[1:-1]):
#         if siglum in vg_node.key.sigla():
#             witness_nodes.append(index)
#     for source, target in zip(witness_nodes, witness_nodes[1:]):
#         a.edge(str(source), str(target), label=str(siglum))

# create dictionary of all edges; key is (source, target) node id, value is list of sigla
all_edges = defaultdict(list)
for w in range(len(witnesses)): # w is offset of witness in list of witnesses
    siglum = 'w' + str(w) # siglum as recorded in SkiplistNode keys
    witness_nodes = []
    for index, vg_node in enumerate(dumpNodes(vg)[1:-1]):
        if siglum in vg_node.key.sigla():
            witness_nodes.append(index)
    for source, target in zip(witness_nodes, witness_nodes[1:]):
        all_edges[(source, target)].append(siglum)

# add edges to graph
for edge_endpoints, sigla in all_edges.items():
    a.edge(str(edge_endpoints[0]), str(edge_endpoints[1]), label=",".join(sigla))

SVG(a.view())

In [11]:
# for index,node in enumerate(dumpNodes(vg)):
#     pp.pprint((index,node))

In [12]:
# list(enumerate(token_array))

In [13]:
pp.pprint(list(enumerate(prioritized_blocks)))

[ ( 0,
    Block(token_count=176, start_position=37, end_position=39, all_start_positions=[234, 467, 703], witnesses={1, 2, 3}, witness_count=3, frequency=3)),
  ( 1,
    Block(token_count=175, start_position=895, end_position=897, all_start_positions=[235, 468, 704], witnesses={1, 2, 3}, witness_count=3, frequency=3)),
  ( 2,
    Block(token_count=174, start_position=401, end_position=403, all_start_positions=[236, 469, 705], witnesses={1, 2, 3}, witness_count=3, frequency=3)),
  ( 3,
    Block(token_count=173, start_position=791, end_position=793, all_start_positions=[237, 470, 706], witnesses={1, 2, 3}, witness_count=3, frequency=3)),
  ( 4,
    Block(token_count=172, start_position=715, end_position=717, all_start_positions=[238, 471, 707], witnesses={1, 2, 3}, witness_count=3, frequency=3)),
  ( 5,
    Block(token_count=171, start_position=369, end_position=371, all_start_positions=[239, 472, 708], witnesses={1, 2, 3}, witness_count=3, frequency=3)),
  ( 6,
    Block(token_count=1

 383,
    Block(token_count=156, start_position=421, end_position=422, all_start_positions=[310, 543], witnesses={1, 2}, witness_count=2, frequency=2)),
  ( 384,
    Block(token_count=155, start_position=237, end_position=238, all_start_positions=[311, 544], witnesses={1, 2}, witness_count=2, frequency=2)),
  ( 385,
    Block(token_count=154, start_position=193, end_position=194, all_start_positions=[312, 545], witnesses={1, 2}, witness_count=2, frequency=2)),
  ( 386,
    Block(token_count=153, start_position=85, end_position=86, all_start_positions=[313, 546], witnesses={1, 2}, witness_count=2, frequency=2)),
  ( 387,
    Block(token_count=19, start_position=125, end_position=128, all_start_positions=[214, 447, 680, 916], witnesses={0, 1, 2, 3}, witness_count=4, frequency=4)),
  ( 388,
    Block(token_count=19, start_position=133, end_position=136, all_start_positions=[158, 391, 624, 860], witnesses={0, 1, 2, 3}, witness_count=4, frequency=4)),
  ( 389,
    Block(token_count=19, star

In [14]:
# for index,block in enumerate(prioritized_blocks):
#     first_token = block.all_start_positions[0]
#     first_five_tokens = token_array[first_token: first_token + 4] # wrong with token_count < 5
#     witnesses = block.witnesses
#     token_count = block.token_count
#     if 'when' in first_five_tokens:
#         pp.pprint((index, first_five_tokens, witnesses, token_count))

In [15]:
# for index,value in enumerate(token_array):
#     if value == 'when':
#         print(token_membership_array[index], token_array[index - 2 : index + 3])

# Start here

1. Add block identifier to VG token when token is created:  store 1) block id, 2) offset within block. This won't track all blocks associated with the token, but it will record the block we were processing when the token was first created.
1. Create hyperedges that group *sibling* (*transposition*) VG nodes as they're created. Decide what to do about them later. They'll tell us which nodes are related, but not necessarily which specifically are transposed with which others.

# The offset within a block is calculated in exact positions in line 17
We would need to record the relative offset next to the block identifier, so the exact position - the start position.

# Priority

1. We already prioritize blocks
1. We can also prioritize tokens with blocks 

In [16]:
# import itertools
# groups = []
# unique_keys = []
# result = itertools.groupby(vg.keys(), key=lambda x: (x.block_id, x.token_offset))
# # pp.pprint(list(result))
# for k, g in result:
#     if k != (None, None):
#             groups.append(list(g))
#             unique_keys.append(k)
# pp.pprint(list(zip(unique_keys, groups)))

# for vg_node in vg:
#     if vg_node.block_id:
#         print(vg_node.block_id, vg_node.token_offset, vg_node)

# groups = []
# uniquekeys = []
# data = sorted(data, key=keyfunc)
# for k, g in groupby(data, keyfunc):
#     groups.append(list(g))      # Store group iterator as a list
#     uniquekeys.append(k)

In [17]:
# Sort vg by block_id and first token offset
# Return block_id, first token offset, and token strings with numbers
# vg_nodes = sorted([(k.block_id, k.token_offset, k) for k in vg.keys() if k.block_id != None], key=lambda x: (x[0], x[1]))
# for vg_node in vg_nodes:
#     key = vg_node[2]
#     print((vg_node[0:2], [token_array[key[i]] for i in ['w0', 'w1', 'w2', 'w3'] if i in key]))

In [18]:
# Low-priority blocks are more likely to be transposed
# Blocks appear to be okay; check how we add them to the variant graph
# for pb in prioritized_blocks:
#     print(set([token_array[sp] for sp in pb.all_start_positions]))

In [19]:
# pp.pprint(list(enumerate(token_array)))

In [20]:
# tuple_to_token_map

In [21]:
# len(prioritized_blocks)

In [22]:
# placed_tokens

In [23]:
witness_sigla

['w0', 'w1', 'w2', 'w3']

In [24]:
vg

SkipList(((w3:879, [879]), (w3:702, [702]), (w3:700, [700]), (w0:91, [91]), (w0:36, [36]), (w0:32, [32]), (w3:701, [701]), (w0:126, [126]), (w0:35, [35]), (w0:34, [34]), (w0:33, [33]), (w0:20, [20]), (w0:12, [12]), (w0:6, [6]), (w0:21, [21]), (w0:7, [7]), (w0:4, [4]), (w0:3, [3]), (w0:17, [17]), (w0:28, [28]), (w0:24, [24]), (w0:13, [13]), (w0:10, [10]), (w0:1, [1]), (w0:30, [30]), (w0:19, [19]), (w0:16, [16]), (w0:15, [15]), (w0:9, [9]), (w0:5, [5]), (w0:27, [27]), (w0:31, [31]), (w0:29, [29]), (w0:26, [26]), (w0:25, [25]), (w0:23, [23]), (w0:22, [22]), (w0:18, [18]), (w0:14, [14]), (w0:11, [11]), (w0:8, [8]), (w0:2, [2]), (w3:703, [703]), (w2:467, [467]), (w1:234, [234]), (w0:0, [0]), (w0:105, [105]), (w0:118, [118]), (w0:116, [116]), (w0:96, [96]), (w0:111, [111]), (w0:123, [123]), (w0:117, [117]), (w0:103, [103]), (w0:121, [121]), (w0:120, [120]), (w0:101, [101]), (w0:97, [97]), (w0:93, [93]), (w0:124, [124]), (w0:122, [122]), (w0:113, [113]), (w0:106, [106]), (w0:104, [104]), (w0:

In [25]:
#  testing
vg = SkipList()
tail_value = len(token_array) + 1
end_node = VG_node(None, None, witness_sigla, w0=tail_value, w1=tail_value, w2=tail_value, w3=tail_value)
vg.insert(VG_node(None, None, witness_sigla, w0=-1, w1=-1, w2=-1, w3=-1), ['[START]'])
print(end_node)
print(end_node.sigla())


vg.insert(end_node, ['[END]'])

w1:936|w0:936|w3:936|w2:936
{'w1', 'w0', 'w3', 'w2'}
{'w1', 'w0', 'w3', 'w2'}
None
{'w1', 'w0', 'w3', 'w2'}
None
